In [1]:
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import pickle
from PIL import Image
import urllib
from adjustText import adjust_text


In [2]:
chromedriver_autoinstaller.install()

import warnings
warnings.filterwarnings('ignore')

## Pull Data

In [3]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640902/Live/England-Premier-League-2022-2023-Chelsea-Fulham"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

Region: England, League: Premier League, Season: 2022/2023, Match Id: 1640902


In [4]:
homeID = home_data['teamId']
awayID = away_data['teamId']

names = {
    homeID : home_name, 
    awayID : away_name
}

## Colours 

In [5]:
home_color = '#2D5DE7'
away_color = '#9C9C9C'

## Pass Field Gains
- Using @SaturdayOnCouch method of Field Gains to assess Progression distance
- Will only consider (a) Positive Field Gains (b) Passes that constitute at least a 10% Field Gain

In [25]:
home_passes = events_df[(events_df['teamId'] == homeID) & 
                        (events_df['type'] == 'Pass') & 
                        (events_df['outcomeType'] == 'Successful')].reset_index(drop=True)[['playerName', 'x', 'y', 'endX', 'endY']]
home_passes['startDist'] = ''
home_passes['endDist'] = ''

for i in range(len(home_passes)):
    x0 = home_passes['x'][i]
    x1 = home_passes['endX'][i]
    y0 = home_passes['y'][i]
    y1 = home_passes['endY'][i]
    
    startDist = np.sqrt((100 - x0)**2 + (50 - y0)**2)
    endDist = np.sqrt((100 - x1)**2 + (50 - y1)**2)
    
    home_passes['startDist'][i] = startDist
    home_passes['endDist'][i] = endDist
    
home_passes['changeDist'] = home_passes['endDist'] - home_passes['startDist']
fwd_passes = home_passes[home_passes['changeDist'] < 0].reset_index(drop=True)
fwd_passes['changeDist'] = abs(fwd_passes['changeDist'])
fwd_passes['pctTwdGoal'] = fwd_passes['changeDist'] / fwd_passes['startDist']
fwd_passes = fwd_passes[fwd_passes['pctTwdGoal'] >= 0.1]
home_field_gains_pass = pd.DataFrame(fwd_passes.groupby('playerName')['pctTwdGoal'].sum()).reset_index()
home_field_gains_pass.rename({'pctTwdGoal':'passFieldGain'}, axis='columns', inplace=True)
home_field_gains_pass

# For Away Team
away_passes = events_df[(events_df['teamId'] == awayID) & 
                        (events_df['type'] == 'Pass') & 
                        (events_df['outcomeType'] == 'Successful')].reset_index(drop=True)[['playerName', 'x', 'y', 'endX', 'endY']]
away_passes['startDist'] = ''
away_passes['endDist'] = ''

for i in range(len(away_passes)):
    x0 = away_passes['x'][i]
    x1 = away_passes['endX'][i]
    y0 = away_passes['y'][i]
    y1 = away_passes['endY'][i]
    
    startDist = np.sqrt((100 - x0)**2 + (50 - y0)**2)
    endDist = np.sqrt((100 - x1)**2 + (50 - y1)**2)
    
    away_passes['startDist'][i] = startDist
    away_passes['endDist'][i] = endDist
    
away_passes['changeDist'] = away_passes['endDist'] - away_passes['startDist']
fwd_passes2 = away_passes[away_passes['changeDist'] < 0].reset_index(drop=True)
fwd_passes2['changeDist'] = abs(fwd_passes2['changeDist'])
fwd_passes2['pctTwdGoal'] = fwd_passes2['changeDist'] / fwd_passes2['startDist']
fwd_passes2 = fwd_passes2[fwd_passes2['pctTwdGoal'] >= 0.1]
away_field_gains_pass = pd.DataFrame(fwd_passes2.groupby('playerName')['pctTwdGoal'].sum()).reset_index()
away_field_gains_pass.rename({'pctTwdGoal':'passFieldGain'}, axis='columns', inplace=True)
away_field_gains_pass

,playerName,passFieldGain
0,Aleksandar Mitrovic,0.675687
1,Andreas Pereira,1.15982
2,Antonee Robinson,1.82165
3,Bernd Leno,3.936537
4,Bobby De Cordova-Reid,3.219499
5,Harrison Reed,1.572323
6,Issa Diop,1.465206
7,João Palhinha,1.682122
8,Kenny Tete,4.348026
9,Manor Solomon,0.537731


## Carry Field Gains 

In [19]:
df_carries = pd.DataFrame()
min_len = 3
max_len = 60
min_time = 1
max_time = 12

for idx, match_event in events_df.iterrows():
    if idx == len(events_df) - 1:
        break
    
    cur_team = match_event['teamId']
    idx_next = idx + 1
    events_df_next = events_df.loc[idx_next]
    take_ons = 0
    loop = True
    count = 0 
    
    while loop:
        next_events = events_df.loc[idx_next]
        
        if next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Successful':
            take_ons += 1
            loop = True
        
        elif (next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['teamId'] != cur_team and next_events['type'] == 'Challenge' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['type'] == 'Foul'):
            loop = True
            
        else:
            loop = False
            
        idx_next += 1
        count += 0 
        

    same_team = cur_team == next_events['teamId']
    not_ball_touch = match_event['type'] != 'BallTouch'
    dx = 105 * (match_event['endX'] - next_events['x'])/100
    dy = 68 * (match_event['endY'] - next_events['y'])/100
    min_dist = dx**2 + dy**2 >= min_len**2
    max_dist = dx**2 + dy**2 <= max_len**2
    same_period = match_event['period'] == next_events['period']
    
    valid_carry = same_team & not_ball_touch & min_dist & max_dist & same_period 
    
    if valid_carry:
        carry = pd.DataFrame()
        prev = match_event
        nxt = next_events
        carry.loc[0, 'eventId'] = prev['eventId'] + count
        carry['id'] = prev['id'] + count
        carry['teamId'] = nxt['teamId']
        carry['x'] = prev['endX']
        carry['y'] = prev['endY']
        carry['period'] = nxt['period']
        carry['type'] = carry.apply(lambda x: {'value': 99, 'displayName': 'Carry'}, axis=1)
        carry['outcomeType'] = 'Successful'
        carry['qualifiers'] = carry.apply(
            lambda x: {'type': {'value': 999, 'displayName': 'takeOns'}, 'value': str(take_ons)}, axis=1)
        carry['satisfiedEventsTypes'] = carry.apply(lambda x: [], axis=1)
        carry['isTouch'] = True
        carry['playerId'] = nxt['playerId']
        carry['playerName'] = nxt['playerName']
        carry['endX'] = nxt['x']
        carry['endY'] = nxt['y']
        
        df_carries = pd.concat([df_carries, carry], ignore_index=True, sort=False)


In [26]:
# Home Team
home_carries = df_carries[df_carries['teamId'] == homeID].reset_index(drop=True)

home_carries['startDist'] = ''
home_carries['endDist'] = ''

for i in range(len(home_carries)):
    x0 = home_carries['x'][i]
    x1 = home_carries['endX'][i]
    y0 = home_carries['y'][i]
    y1 = home_carries['endY'][i]
    
    startDist = np.sqrt((100 - x0)**2 + (50 - y0)**2)
    endDist = np.sqrt((100 - x1)**2 + (50 - y1)**2)
    
    home_carries['startDist'][i] = startDist
    home_carries['endDist'][i] = endDist
    
home_carries['changeDist'] = home_carries['endDist'] - home_carries['startDist']
fwd_carries = home_carries[home_carries['changeDist'] < 0].reset_index(drop=True)
fwd_carries['changeDist'] = abs(fwd_carries['changeDist'])
fwd_carries['pctTwdGoal'] = fwd_carries['changeDist'] / fwd_carries['startDist']
fwd_carries = fwd_carries[fwd_carries['pctTwdGoal'] >= 0.1]
home_field_gains_carries = pd.DataFrame(fwd_carries.groupby('playerName')['pctTwdGoal'].sum()).reset_index()
home_field_gains_carries.rename({'pctTwdGoal':'carryFieldGain'}, axis='columns', inplace=True)
home_field_gains_carries


# Away Team
away_carries = df_carries[df_carries['teamId'] == awayID].reset_index(drop=True)

away_carries['startDist'] = ''
away_carries['endDist'] = ''

for i in range(len(away_carries)):
    x0 = away_carries['x'][i]
    x1 = away_carries['endX'][i]
    y0 = away_carries['y'][i]
    y1 = away_carries['endY'][i]
    
    startDist = np.sqrt((100 - x0)**2 + (50 - y0)**2)
    endDist = np.sqrt((100 - x1)**2 + (50 - y1)**2)
    
    away_carries['startDist'][i] = startDist
    away_carries['endDist'][i] = endDist
    
away_carries['changeDist'] = away_carries['endDist'] - away_carries['startDist']
fwd_carries = away_carries[away_carries['changeDist'] < 0].reset_index(drop=True)
fwd_carries['changeDist'] = abs(fwd_carries['changeDist'])
fwd_carries['pctTwdGoal'] = fwd_carries['changeDist'] / fwd_carries['startDist']
fwd_carries = fwd_carries[fwd_carries['pctTwdGoal'] >= 0.1]
away_field_gains_carries = pd.DataFrame(fwd_carries.groupby('playerName')['pctTwdGoal'].sum()).reset_index()
away_field_gains_carries.rename({'pctTwdGoal':'carryFieldGain'}, axis='columns', inplace=True)
away_field_gains_carries

,playerName,carryFieldGain
0,Aleksandar Mitrovic,0.419226
1,Andreas Pereira,0.27456
2,Antonee Robinson,0.189263
3,Bobby De Cordova-Reid,1.130227
4,Harrison Reed,1.039037
5,Issa Diop,0.714615
6,Kenny Tete,0.451062
7,Manor Solomon,0.244473
8,Tim Ream,0.955363
9,Willian,1.435558


## Merge Datasets

In [28]:
df_home = (home_field_gains_pass.merge(home_field_gains_carries, on='playerName', how='outer')
                                .fillna(0)
          )
df_home


df_away = (away_field_gains_pass.merge(away_field_gains_carries, on='playerName', how='outer')
                                .fillna(0)
          )
df_away

,playerName,passFieldGain,carryFieldGain
0,Aleksandar Mitrovic,0.675687,0.419226
1,Andreas Pereira,1.159820,0.274560
2,Antonee Robinson,1.821650,0.189263
3,Bernd Leno,3.936537,0.000000
4,Bobby De Cordova-Reid,3.219499,1.130227
5,Harrison Reed,1.572323,1.039037
6,Issa Diop,1.465206,0.714615
7,João Palhinha,1.682122,0.000000
8,Kenny Tete,4.348026,0.451062
9,Manor Solomon,0.537731,0.244473


## Plot 

In [43]:
fig, ax = plt.subplots(figsize = (20,15))
fig.set_facecolor("#2B2B2B")
ax.patch.set_facecolor("#2B2B2B")
mpl.rcParams['figure.dpi'] = 800


# Home Team
plt.scatter(df_home.carryFieldGain, df_home.passFieldGain, color=home_color, s=40)

text = []
for i, row in df_home.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['carryFieldGain']
    y = row['passFieldGain']
    sf = float(x+y)
    t = plt.text(x = x+0.0005, y = y+0.002, s = p1.upper(), 
                 bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(14, min(40, 7*(x+y))))
    text.append(t)
    

# Away Team
plt.scatter(df_away.carryFieldGain, df_away.passFieldGain, color=away_color, s=40)

for i, row in df_away.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['carryFieldGain']
    y = row['passFieldGain']
    sf = float(x+y)
    t = plt.text(x = x+0.0005, y = y+0.002, s = p1.upper(), 
                 bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(14, min(40, 7*(x+y))))
    text.append(t)
    
plt.title('\nFields Gained \n\n'.upper(), 
          loc='left', color='w', fontname = 'Sans Serif', fontsize = 30, y=1)
plt.suptitle(t = home_name.upper() + ' ' +score + ' ' + away_name.upper() + '  |  ' + date_str.upper() + '  |  '  + 'CREATED BY ...', x=0.125, y=0.93, ha='left', 
             fontsize=24, color='w')
plt.xlabel('\nFields Gained via Carry', color='w', fontsize=30)
plt.ylabel('Fields Gained via Pass\n', color='w', fontsize=30)

ax.set_axisbelow(True)
ax.yaxis.grid(color='#848484', linestyle='dashed')
ax.xaxis.grid(color='#848484', linestyle='dashed')
# spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
# ticks
for tick in ax.xaxis.get_ticklines(): 
    tick.set_color('w')
for tick in ax.yaxis.get_ticklines(): 
    tick.set_color('w')
plt.xticks(color='w')
plt.yticks(color='w')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

adjust_text(text, ax=ax)

plt.figtext(0.125, -0.0175, "> Based on Fields Gained by @SaturdayOnCouch\n> Includes positive field gains only, i.e. not penalised for negative field gains \n> Includes only actions that constitute at least 10% of a field gained", ha="left", fontsize=16, fontname='Sans Serif', color='white')



fig.savefig('progDist.png', 
            bbox_inches="tight",
            edgecolor="none",
            facecolor = "#2B2B2B",
            dpi=500)